<a href="https://colab.research.google.com/github/agalvare/reportportal/blob/master/val_bot_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
import numpy as np


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

In [6]:
# df = pd.read_excel(r'/content/drive/MyDrive/validation-bot-dataset.xlsx')


In [7]:

questions = ['What is your name?',
 'What is a machine check?',
 'What are the steps to debug a machine check?']
responses = ['My name is amazing bot.',
 "A machine check refers to an error or fault detected by a computer's hardware during its operation. When a computer detects a machine check error, it typically triggers an interrupt, which is a signal that halts the current operation and allows the system to gather more information about the error.\n\nMachine checks can be caused by a variety of factors, including hardware defects, electrical interference, and software errors. Examples of machine check errors include memory errors, CPU errors, bus errors, and input/output errors.\n\nWhen a machine check error occurs, the system will typically log the error and attempt to diagnose the cause of the error. Depending on the severity of the error and the system's configuration, the system may also attempt to recover from the error by retrying the operation or by shutting down the system to prevent further damage.",
 "Debugging a machine check can be a complex process that requires technical expertise and specialized tools. The specific steps to debug a machine check can vary depending on the type of error and the hardware and software involved. However, here are some general steps that can be followed:\n\nCollect system logs: When a machine check error occurs, the system will typically log information about the error, including the type of error, the location of the error, and other relevant details. Collecting these logs is the first step in diagnosing the error.\n\nAnalyze system configuration: Understanding the system's configuration, including the hardware and software components, can help identify potential sources of the error. Check for any recent hardware or software changes that may have caused the error.\n\nCheck hardware components: The error may be caused by a malfunctioning hardware component, such as the memory or the CPU. Running diagnostic tests on the hardware can help identify the faulty component.\n\nCheck for software issues: The error may also be caused by software issues, such as driver conflicts or operating system bugs. Analyzing software logs and updating drivers and software can help resolve these issues.\n\nContact technical support: If the error persists despite the above steps, contact the hardware or software vendor's technical support for further assistance.\n\nIt's important to note that debugging a machine check can be a complicated process that requires specialized knowledge and tools. In some cases, it may be necessary to engage with a hardware or software specialist to resolve the issue."]


In [8]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(questions)
sequences = tokenizer.texts_to_sequences(questions)

In [9]:
tokenizer.word_index

{'what': 1,
 'is': 2,
 'a': 3,
 'machine': 4,
 'check': 5,
 'your': 6,
 'name': 7,
 'are': 8,
 'the': 9,
 'steps': 10,
 'to': 11,
 'debug': 12}

In [32]:
# padded = pad_sequences(sequences, padding='post', maxlen=128, truncating='post') / pad_sequences(sequences, padding='post', maxlen=128, truncating='post').max()
padded = pad_sequences(sequences, padding='post', maxlen=128, truncating='post') / 12
padded[0]*12

array([1., 2., 6., 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
model = Sequential()

# model.add(tf.keras.layers.Flatten(input_shape=(128,),name="question_input"))
model.add(Dense(128, input_shape=(len(padded[0]),), activation='relu',name="question_input"))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(padded[0]), activation='softmax',name="response_output"))
model.summary()
model.compile(loss='categorical_crossentropy',
  metrics=["accuracy"]
              )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 question_input (Dense)      (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 response_output (Dense)     (None, 128)               8320      
                                                                 
Total params: 33,088
Trainable params: 33,088
Non-trainable params: 0
_________________________________________________________________


In [12]:
global is_ok
is_ok = False
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    global is_ok
    # print(logs.get('loss'),"<-------------")
    if(logs.get('accuracy')>0.95):
      is_ok = True
      print(f"\nReached {logs.get('accuracy')*100} accuracy so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()
hist = model.fit(
    x = np.array(padded),
    y = np.array(padded),
    epochs=1000, 
    batch_size=5, 
    verbose=0,
    callbacks = [callbacks]
    )



Reached 100.0 accuracy so cancelling training!


In [13]:
if not is_ok:
  exit()

In [31]:
query = "What is your name"
tokenizer2 = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer2.fit_on_texts(questions)
sequences = tokenizer2.texts_to_sequences([query])
sequences

[[1, 2, 6, 7]]

In [14]:
classifications = model.predict(np.array(padded))

1/1 [==============================] - 0s 396ms/step


In [ ]:
ERROR_THESHOLD = 0.25
results = [[i, r] for i, r in enumerate(res) if r > ERROR_THESHOLD]
results.sort(key=lambda x: x[1], reverse=True)

In [15]:
classifications[1]*12

array([1.9937795e-01, 1.9774200e+00, 1.6641258e+00, 4.4887629e+00,
       3.0294728e+00, 3.3146942e-01, 8.1736416e-02, 1.9359237e-01,
       3.4042735e-02, 1.0332225e-14, 4.1047047e-15, 1.5303491e-13,
       2.5668638e-13, 1.8385136e-14, 3.8389898e-14, 2.4005829e-14,
       4.8543759e-13, 1.6014650e-14, 4.0637625e-14, 9.7175643e-15,
       4.3871919e-14, 3.2442056e-14, 2.0797674e-14, 1.8326110e-14,
       3.4031294e-14, 1.3031800e-13, 1.3216624e-15, 1.9363179e-13,
       2.1207306e-14, 2.7708044e-12, 8.9802569e-14, 1.4138379e-14,
       6.3501049e-13, 1.0602535e-14, 6.6699589e-14, 1.1056191e-13,
       1.9456167e-14, 5.8844558e-14, 5.1984445e-14, 1.2901710e-13,
       3.5408669e-15, 2.7564837e-15, 2.7643930e-14, 1.6974373e-14,
       2.1523624e-13, 2.2904264e-13, 7.1071621e-15, 9.4348818e-14,
       9.5272424e-16, 8.0233587e-15, 4.8140518e-14, 6.0429466e-13,
       3.5181395e-15, 8.9142772e-15, 2.6809467e-14, 1.1539491e-15,
       6.8682717e-14, 1.3852335e-14, 4.1205703e-13, 4.2416655e

In [16]:
predicted_words = []
for i in range(len(classifications)):
  predicted_words.append(tokenizer.index_word[np.argmax(classifications[i])])
print(predicted_words)

['a', 'a', 'a']


In [17]:
# model.save(rf'100chatbot_model.h5', hist)
# print("Done")